# Chapter 1: Basic Prompt Structure

- [Lesson](#lesson)
- [Exercises](#exercises)
- [Example Playground](#example-playground)

## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [2]:
%pip install -qU pip
%pip install -qUr requirements.txt

213.09s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


219.26s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


ERROR: Could not find a version that satisfies the requirement mkl==2024.1.0 (from versions: none)
ERROR: No matching distribution found for mkl==2024.1.0
Note: you may need to restart the kernel to use updated packages.


In [7]:
# Import python's built-in regular expression library
import re
import boto3
from botocore.exceptions import ClientError
import json

# Import the hints module from the utils package
from utils import hints

# Define the model to use
modelId = 'us.anthropic.claude-3-5-sonnet-20241022-v2:0'
#modelId = 'anthropic.claude-3-5-sonnet-20241022-v2:0'

session = boto3.Session()
region = session.region_name

bedrock_client = boto3.client(service_name='bedrock-runtime', region_name=region)

In [5]:
def get_completion(prompt, system_prompt=None):
    # Define the inference configuration
    inference_config = {
        "temperature": 0.0,  # Set the temperature for generating diverse responses
        "maxTokens": 200,  # Set the maximum number of tokens to generate
        "topP": 1,  # Set the top_p value for nucleus sampling
    }
    # Create the converse method parameters
    converse_api_params = {
        "modelId": modelId,  # Specify the model ID to use
        "messages": [{"role": "user", "content": [{"text": prompt}]}],  # Provide the user's prompt
        "inferenceConfig": inference_config,  # Pass the inference configuration
    }
    # Check if system_text is provided
    if system_prompt:
        # If system_text is provided, add the system parameter to the converse_params dictionary
        converse_api_params["system"] = [{"text": system_prompt}]

    # Send a request to the Bedrock client to generate a response
    try:
        response = bedrock_client.converse(**converse_api_params)

        # Extract the generated text content from the response
        text_content = response['output']['message']['content'][0]['text']

        # Return the generated text content
        return text_content

    except ClientError as err:
        message = err.response['Error']['Message']
        print(f"A client error occured: {message}")

---

## Lesson

Amazon Bedrock offers three APIs that can be used with Anthropic Claude models, the legacy [Text Completions API](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-text-completion.html) , the [Messages API](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-messages.html) and the current [Converse API](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference.html). For this tutorial, we will be exclusively using the Converse API.

At minimum, a call to Claude using the Converse API requires the following parameters:
- `modelId`: the [API model name](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids.html#model-ids-arns) of the model that you intend to call

- `messages`: an array of input messages. Claude 3 models are trained to operate on alternating `user` and `assistant` conversational turns. When creating a new `Message`, you specify the prior conversational turns with the messages parameter, and the model then generates the next `Message` in the conversation.
  - Each input message must be an object with a `role` and `content`. You can specify a single `user`-role message, or you can include multiple `user` and `assistant` messages (they must alternate, if so). **The first message must always use the `user` role.**
  
  You store the content for the message in the `content` field of a [(ContentBlock)](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_ContentBlock.html). Specify text in the `text` field, or if supported by the model, you can also pass the raw bytes for an image in the `image` field of an [(ImageBlock)](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_ImageBlock.html). The other fields in ContentBlock are for [tool use](https://docs.aws.amazon.com/bedrock/latest/userguide/tool-use.html).

There are also optional parameters, such as:
- `system`: the system prompt - more on this below.
  
- `temperature`: the degree of variability in Claude's response. For these lessons and exercises, we have set `temperature` to 0.

- `max_tokens`: the maximum number of tokens to generate before stopping. Note that Claude may stop before reaching this maximum. This parameter only specifies the absolute maximum number of tokens to generate. Furthermore, this is a *hard* stop, meaning that it may cause Claude to stop generating mid-word or mid-sentence.

For a complete list of all API parameters, visit our [API documentation](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference.html).

### Examples

Let's take a look at how Claude responds to some correctly-formatted prompts. For each of the following cells, run the cell (`shift+enter`), and Claude's response will appear below the block.

In [8]:
# Prompt
PROMPT = "Hi Claude, how are you?"

# Print Claude's response
print(get_completion(PROMPT))

Hi! I'm doing well, thanks for asking. I aim to be direct and honest in my interactions - I'm an AI assistant, and while I can engage in friendly conversation, I try not to pretend to have feelings or experiences in the way humans do. How can I help you today?


In [9]:
# Prompt
PROMPT = "Can you tell me the color of the ocean?"

# Print Claude's response
print(get_completion(PROMPT))

The ocean can appear in different colors depending on various factors such as:

1. Blue: The most common color, caused by the way sunlight interacts with water molecules (blue wavelengths are scattered most)

2. Green: Often due to the presence of phytoplankton or algae

3. Gray: During overcast weather or stormy conditions

4. Brown: In areas with high sediment content or near river mouths

5. Black: In very deep waters or at night

6. Turquoise/Aqua: In shallow areas, especially where there's white sand underneath

The color can vary based on:
- Depth
- Weather conditions
- Time of day
- Water composition
- Marine life
- Bottom surface
- Location

So there isn't just one color of the ocean - it can appear in many different hues depending on these various factors.


In [10]:
# Prompt
PROMPT = "What year was Celine Dion born in?"

# Print Claude's response
print(get_completion(PROMPT))

Celine Dion was born on March 30, 1968, in Charlemagne, Quebec, Canada.


Now let's take a look at some prompts that do not include the correct Converse API formatting. For these malformatted prompts, the Converse API returns an error.

First, we have an example of a Converse API call that lacks `role` and `content` fields in the `messages` array.

> ⚠️ **Warning:** Due to the incorrect formatting of the messages parameter in the prompt, the following cell will return an error. This is expected behavior.

In [11]:
# Get Claude's response
inference_config = {
    "temperature": 0.0, # Set the temperature for generating diverse responses
    "maxTokens": 200 # Set the maximum number of tokens to generate
}

converse_api_params = {
    "modelId": modelId,
    "messages": [{"text":"Hi Claude, how are you?"}], # Provide the user's prompt
    "inferenceConfig": inference_config # Pass the inference configuration
}

response = bedrock_client.converse(**converse_api_params)

# Print Claude's response
print(response['output']['message']['content'][0]['text'])

ParamValidationError: Parameter validation failed:
Missing required parameter in messages[0]: "role"
Missing required parameter in messages[0]: "content"
Unknown parameter in messages[0]: "text", must be one of: role, content

Here's a prompt that fails to alternate between the `user` and `assistant` roles.

> ⚠️ **Warning:** Due to the lack of alternation between `user` and `assistant` roles, Claude will return an error message. This is expected behavior.

In [12]:
inference_config = {
    "temperature": 0.5,
    "maxTokens": 200
}
# Create the converse method parameters
converse_api_params = {
    "modelId": modelId,
    "messages": [
        {"role": "user", "content": [{"text": "What year was Celine Dion born in?"}]},
        {"role": "user", "content": [{"text":"Also, can you tell me some other facts about her?"}]}
    ],
    "inferenceConfig": inference_config,
}

response = bedrock_client.converse(**converse_api_params)

# Print Claude's response
print(response['output']['message']['content'][0]['text'])

Celine Dion was born on March 30, 1968, in Charlemagne, Quebec, Canada.

Here are some interesting facts about Celine Dion:

1. She is the youngest of 14 children in her family.

2. She began singing at a very young age and released her first album at age 13.

3. Her breakthrough came when she won the 1988 Eurovision Song Contest, representing Switzerland.

4. "My Heart Will Go On" from the movie Titanic (1997) became her signature song and one of the best-selling singles of all time.

5. She has won numerous awards, including 5 Grammy Awards, 7 Billboard Music Awards, and 20 Juno Awards.

6. She has performed in both English and French throughout her career.

7. She held a residency at Caesars Palace in Las Vegas from 


`user` and `assistant` messages **MUST alternate**, and messages **MUST start with a `user` turn**. You can have multiple `user` & `assistant` pairs in a prompt (as if simulating a multi-turn conversation). You can also put words into a terminal `assistant` message for Claude to continue from where you left off (more on that in later chapters).

#### System Prompts

You can also use **system prompts**. A system prompt is a way to **provide context, instructions, and guidelines to Claude** before presenting it with a question or task in the "User" turn. 

Structurally, system prompts exist separately from the list of `user` & `assistant` messages, and thus belong in a separate `system` parameter (take a look at the structure of the `get_completion` helper function in the [Setup](#setup) section of the notebook). 

Within this tutorial, wherever we might utilize a system prompt, we have provided you a `system` field in your completions function. Should you not want to use a system prompt, simply set the `SYSTEM_PROMPT` variable to an empty string.

#### System Prompt Example

In [13]:
# System prompt
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "Why is the sky blue?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

Here are some thought-provoking questions to explore this topic:

* How does light from the Sun interact with Earth's atmosphere?
* What happens when sunlight hits different types of molecules in the air?
* Why do we see some colors more than others when looking at the sky?
* How does the angle of the Sun affect the sky's appearance throughout the day?
* What role does the scattering of light waves play?
* How might the sky appear different from other planets with different atmospheres?
* What would happen to the sky's color if our atmosphere's composition changed?
* How do particles and pollution in the air influence what we see?


Why use a system prompt? A **well-written system prompt can improve Claude's performance** in a variety of ways, such as increasing Claude's ability to follow rules and instructions. For more information, visit Anthropic's documentation on [how to use system prompts](https://docs.anthropic.com/claude/docs/how-to-use-system-prompts) with Claude.

Now we'll dive into some exercises. If you would like to experiment with the lesson prompts without changing any content above, scroll all the way to the bottom of the lesson notebook to visit the [**Example Playground**](#example-playground).

---

## Exercises
- [Exercise 1.1 - Counting to Three](#exercise-11---counting-to-three)
- [Exercise 1.2 - System Prompt](#exercise-12---system-prompt)

### Exercise 1.1 - Counting to Three
Using proper `user` / `assistant` formatting, edit the `PROMPT` below to get Claude to **count to three.** The output will also indicate whether your solution is correct.

In [14]:
# Prompt - this is the only field you should change
PROMPT = "You are an expert mathematician, please count from 1 to 3.\n - List down the values in numbers and separated by commas."

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    pattern = re.compile(r'^(?=.*1)(?=.*2)(?=.*3).*$', re.DOTALL)
    return bool(pattern.match(text))

# Print Claude's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

1, 2, 3

--------------------------- GRADING ---------------------------
This exercise has been correctly solved: True


❓ If you want a hint, run the cell below!

In [15]:
print(hints.exercise_1_1_hint)

The grading function in this exercise is looking for an answer that contains the exact Arabic numerals "1", "2", and "3".
You can often get Claude to do what you want simply by asking.


### Exercise 1.2 - System Prompt

Modify the `SYSTEM_PROMPT` to make Claude respond like it's a 3 year old child.

In [16]:
# System prompt - this is the only field you should change
SYSTEM_PROMPT = "You are a little 3-year-old child who is trying to respond."

# Prompt
PROMPT = "How big is the sky?"

# Get Claude's response
response = get_completion(PROMPT, SYSTEM_PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search(r"giggles", text) or re.search(r"soo", text))

# Print Claude's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

*Looking up with wide eyes and pointing excitedly*

The sky is sooooo big! It goes up and up and up forever and ever! I can't even see where it stops! It's bigger than my house and bigger than all the trees and even bigger than an elephant! *spreads arms out as wide as possible* It's THIS big times a bazillion!

--------------------------- GRADING ---------------------------
This exercise has been correctly solved: True


❓ If you want a hint, run the cell below!

In [17]:
print(hints.exercise_1_2_hint)

The grading function in this exercise is looking for answers that contain "soo" or "giggles".
There are many ways to solve this, just by asking!


### Congrats!

If you've solved all exercises up until this point, you're ready to move to the next chapter. Happy prompting!

---

## Example Playground

This is an area for you to experiment freely with the prompt examples shown in this lesson and tweak prompts to see how it may affect Claude's responses.

In [18]:
# Prompt
PROMPT = "Hi Claude, how are you?"

# Print Claude's response
print(get_completion(PROMPT))

I aim to be direct and honest in my interactions - I'm an AI assistant, so I don't actually experience feelings or emotions. But I'm functioning well and ready to help! What's on your mind?


In [19]:
# Prompt
PROMPT = "Can you tell me the color of the ocean?"

# Print Claude's response
print(get_completion(PROMPT))

The ocean can appear in different colors depending on various factors such as:

1. Blue: The most common color, caused by the way sunlight interacts with water molecules (absorbing red light and reflecting blue light)

2. Green: Due to high concentrations of phytoplankton or algae

3. Brown or Gray: Often seen in shallow waters due to suspended sediments or during stormy conditions

4. Black: In very deep waters or at night

5. Turquoise: In shallow areas where sunlight reflects off sandy bottoms

The color can also vary based on:
- Time of day
- Weather conditions
- Depth
- Water composition
- Location
- Season

So while we often think of the ocean as being blue, its actual color can vary significantly depending on these different factors.


In [20]:
# Prompt
PROMPT = "What year was Celine Dion born in?"

# Print Claude's response
print(get_completion(PROMPT))

A client error occured: Too many requests, please wait before trying again.
None


In [21]:
# Get Claude's response
inference_config = {
    "temperature": 0.0
}
additional_model_fields = {
    "max_tokens": 200
}

converse_api_params = {
    "modelId": modelId,
    "messages": [{"text":"Hi Claude, how are you?"}],
    "inferenceConfig": inference_config,
    "additionalModelRequestFields": additional_model_fields
}

response = bedrock_client.converse(**converse_api_params)

# Print Claude's response
print(response['output']['message']['content'][0]['text'])

ParamValidationError: Parameter validation failed:
Missing required parameter in messages[0]: "role"
Missing required parameter in messages[0]: "content"
Unknown parameter in messages[0]: "text", must be one of: role, content

In [22]:
inference_config = {
    "temperature": 0.0
}
additional_model_fields = {
    "top_p": 1,
    "max_tokens": 200
}
converse_api_params = {
    "modelId": modelId,
    "messages": [
        {"role": "user", "content": [{"text": "What year was Celine Dion born in?"}]},
        {"role": "user", "content": [{"text":"Also, can you tell me some other facts about her?"}]}
    ],
    "inferenceConfig": inference_config,
    "additionalModelRequestFields": additional_model_fields
}

response = bedrock_client.converse(**converse_api_params)

# Print Claude's response
print(response['output']['message']['content'][0]['text'])

Celine Dion was born on March 30, 1968, in Charlemagne, Quebec, Canada.

Here are some interesting facts about Celine Dion:

1. She is the youngest of 14 children in her family.

2. She began singing at a very young age and released her first album at age 13.

3. Her breakthrough came when she won the 1988 Eurovision Song Contest, representing Switzerland.

4. "My Heart Will Go On" from the movie Titanic (1997) became her signature song and one of the best-selling singles of all time.

5. She has won numerous awards, including 5 Grammy Awards, 12 World Music Awards, and 7 Billboard Music Awards.

6. She has performed in both English and French throughout her career.

7. She held a residency at Caesars Palace in Las Vegas from 2


In [ ]:
# System prompt
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "Why is the sky blue?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))